# Spoonacualr

In [1]:
!poetry add openapi-pydantic



The following packages are already present in the pyproject.toml and will be skipped:

  • openapi-pydantic

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [1]:

from langchain.agents import AgentType, initialize_agent
from langchain.requests import Requests
from langchain_community.agent_toolkits import NLAToolkit
from langchain_openai import OpenAI
# from langchain_community 

spoonacular_api_key = "a64f3bca257a4bdf8341a607335fdeb1"

from langchain_community.llms import Ollama

llm = Ollama(model='mistral:7b-instruct-v0.2-q6_K')




In [2]:
from langchain_community.llms import Ollama

# llm = Ollama(model='llama2:13b-chat-q4_K_S')


requests = Requests(headers={"x-api-key": spoonacular_api_key})
spoonacular_toolkit = NLAToolkit.from_llm_and_url(
    llm,
    "https://spoonacular.com/application/frontend/downloads/spoonacular-openapi-3.json",
    requests=requests,
    max_text_length=1000,  # If you want to truncate the response text
)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Accept. Valid values are ['query', 'path'] Ig

In [3]:
api_tools = spoonacular_toolkit.get_tools()

text_to_remove = ['ByID', 'Information', 'Image', 'visualize', 'Analyzed', 'create', 'Website', 'Widget', 'Bulk', 'map', 'Week', 'delete', 'add', 'User', 'Videos', 'talk', 'Pairing', 'Restaurants', 'quickAnswer', 'ConversationSuggests', 'Templates', 'MenuItemSearch', 'SiteContent']

# filtered_sample = [item for item in sample if text_to_remove not in item.name]

filtered_api_tools = [item for item in api_tools if all(text not in item.name for text in text_to_remove)]

Tools

value_when_true if condition else value_when_false

In [4]:
image_url_tool = None
 

for tool in filtered_api_tools:
    print(tool.name)
    
       
    # image_url_tool = tool if tool.name == 'spoonacular_API.imageAnalysisByURL'
    if tool.name == 'spoonacular_API.imageAnalysisByURL':
        image_url_tool = tool
        
    # if tool.name == 'spoonacular_API.imageAnalysisByURL':
    #     image_url_tool = tool
    
        

spoonacular_API.searchRecipes
spoonacular_API.searchRecipesByIngredients
spoonacular_API.searchRecipesByNutrients
spoonacular_API.getSimilarRecipes
spoonacular_API.getRandomRecipes
spoonacular_API.autocompleteRecipeSearch
spoonacular_API.analyzeRecipe
spoonacular_API.summarizeRecipe
spoonacular_API.analyzeRecipeInstructions
spoonacular_API.classifyCuisine
spoonacular_API.analyzeARecipeSearchQuery
spoonacular_API.convertAmounts
spoonacular_API.parseIngredients
spoonacular_API.guessNutritionByDishName
spoonacular_API.computeIngredientAmount
spoonacular_API.computeGlycemicLoad
spoonacular_API.autocompleteIngredientSearch
spoonacular_API.ingredientSearch
spoonacular_API.getIngredientSubstitutes
spoonacular_API.searchGroceryProducts
spoonacular_API.searchGroceryProductsByUPC
spoonacular_API.searchCustomFoods
spoonacular_API.getComparableProducts
spoonacular_API.autocompleteProductSearch
spoonacular_API.classifyGroceryProduct
spoonacular_API.searchMenuItems
spoonacular_API.generateMealPlan
s

In [5]:
image_url_tool

NLATool(name='spoonacular_API.imageAnalysisByURL', description="I'm an AI from spoonacular API. Instruct what you want, and I'll assist via an API with description: Analyze a food image. The API tries to classify the image, guess the nutrition, and find a matching recipes.", func=<bound method Chain.run of OpenAPIEndpointChain(api_request_chain=APIRequesterChain(prompt=PromptTemplate(input_variables=['instructions'], output_parser=APIRequesterOutputParser(), partial_variables={'schema': '/* Analyze a food image. The API tries to classify the image, guess the nutrition, and find a matching recipes. */\ntype imageAnalysisByURL = (_: {\n/* The URL of the image to be analyzed. */\n\t\timageUrl: string,\n}) => any;'}, template='You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user\'s instructions.\n\nAPI_SCHEMA: ```typescript\n{schema}\n```\n\nUSER_INSTRUCTIONS: "{instructions}"\n\nYour arguments must be plain json provided in a markdown block:\n\nAR

In [6]:
type(filtered_api_tools)

list

Agent

In [7]:
from crewai import Agent 
from textwrap import dedent
from langchain_community.chat_models import ChatOllama 
from langchain_community.llms.ollama import Ollama


class ChefAgents:
    def __init__(self):
        self.llm = Ollama(model="mistral:7b-instruct-v0.2-q6_K")      

    def chef_agent(self):
        return Agent(
            role="Master Chef",
            backstory=dedent(
                f"""Expert in Multicuisines and receipies. 
                I have decades of expereince making different different dishes."""),
            goal=dedent(f"""
                        provide the right right receipe for the given dish
                        """),
            tools=filtered_api_tools,
            verbose=True,
            llm=self.llm,
        )

    # def city_selection_expert(self):
    #     return Agent(
    #         role="City Selection Expert",
    #         backstory=dedent(
    #             f"""Expert at analyzing travel data to pick ideal destinations"""),
    #         goal=dedent(
    #             f"""Select the best cities based on weather, season, prices, and traveler interests"""),
    #         tools=[SearchTools.search_internet],
    #         verbose=True,
    #         llm=self.llm,
    #     )

    # def local_tour_guide(self):
    #     return Agent(
    #         role="Local Tour Guide",
    #         backstory=dedent(f"""Knowledgeable local guide with extensive information
    #     about the city, it's attractions and customs"""),
    #         goal=dedent(
    #             f"""Provide the BEST insights about the selected city"""),
    #         tools=[SearchTools.search_internet],
    #         verbose=True,
    #         llm=self.llm,
    #     )



Task

In [8]:
from crewai import Task
from textwrap import dedent



class ChefTasks:
    def __tip_section(self):
        return "If you do your BEST WORK, I'll give you a $10,000 commission!"

    def receipe_generation(self, agent, dish):
        return Task(
            description=dedent(
                f"""
            **Task**: find and provide the appropriate receipe for the dish.
            **Description**: provide the proper receipe for the given dish with accurate measurements if you can.
            
            Outline structure:

            - Dish: "Name of the dish"
            - Ingredients: give the needed ingridients with measurements
            - Step-by-Step Instructions on cooking
            - add any general tips to improve the taste or best practis
            
            **Parameters**: 
            - dish: {dish}
        """
            ),
            agent=agent,
        )
    def ingriedient_finder(self, agent, image_url):
        return Task(
            description=dedent(
                f"""
            **Task**: find the ingriedient in the provided image url.
            **Description**: analyze the image and findout what is in the image, most probably ingredients that can be used for cooking like, vegetables, fruits, meat and so on.
            once you found the items in the image provide the details of it like what was in the image            
            **Parameters**: 
            - image_url: {image_url}
        """
            ),
            agent=agent,
        )


main

In [9]:
# from crewai import Crew
# from textwrap import dedent
# # from agents import TravelAgents

# from dotenv import load_dotenv
# load_dotenv()


# class ChefCrew:
#     def __init__(self, dish, image_url):
#         self.dish = dish
#         self.image_url = image_url

#     def run(self):
#         # Define your custom agents and tasks in agents.py and tasks.py
#         agents = ChefAgents()
#         tasks = ChefTasks()

#         # Define your custom agents and tasks here
#         expert_chef_agent = agents.chef_agent()
    

#         # Custom tasks include agent name and variables as input
#         receipe_task = tasks.receipe_generation(
#             expert_chef_agent,
#             self.dish,
#         )
#         ingridient_task = tasks.ingriedient_finder(
#         expert_chef_agent,
#         self.image_url,
#         )

#         # Define your custom crew here
#         crew = Crew(
#             agents=[expert_chef_agent,
#                     ],
#             tasks=[
#                 receipe_task,
#                 ingridient_task
#             ],
#             verbose=True,
#         )

#         result = crew.kickoff()
#         return result


# # This is the main function that you will use to run your custom crew.
# if __name__ == "__main__":
#     print("## Welcome to Receipe Crew")
#     print('-------------------------------')
#     dish = input(
#         dedent("""
#       what receipe do you need to know ?
#     """))
   
#     trip_crew = ChefCrew(dish)
#     result = trip_crew.run()
#     print("\n\n########################")
#     print(f"## Here is you Receipe for {dish}")
#     print("########################\n")
#     print(result)


In [18]:
from crewai import Crew
from textwrap import dedent
# from agents import TravelAgents
from crewai.process import Process
from dotenv import load_dotenv
load_dotenv()


dish = 'chicken biryani'

        # Define your custom agents and tasks in agents.py and tasks.py
agents = ChefAgents()
tasks = ChefTasks()

        # Define your custom agents and tasks here
expert_chef_agent = agents.chef_agent()
    

        # Custom tasks include agent name and variables as input
receipe_task = tasks.receipe_generation(
    expert_chef_agent,
    dish,
)
ingridient_task = tasks.ingriedient_finder(
expert_chef_agent,
dish,
)

        # Define your custom crew here
crew = Crew(
    agents=[expert_chef_agent,
            ],
    tasks=[
        receipe_task,
        # ingridient_task
    ],
    verbose=True,
#     process=Process.hierarchical,
    manager_llm=llm
)

result = crew.kickoff()
# return result


# This is the main function that you will use to run your custom crew.
# if __name__ == "__main__":
#     print("## Welcome to Receipe Crew")
#     print('-------------------------------')
#     dish = input(
#         dedent("""
#       what receipe do you need to know ?
#     """))
   
#     trip_crew = ChefCrew(dish)
#     result = trip_crew.run()
#     print("\n\n########################")
#     print(f"## Here is you Receipe for {dish}")
#     print("########################\n")
#     print(result)


Working Agent: Master Chef
Starting Task: 
**Task**: find and provide the appropriate receipe for the dish.
**Description**: provide the proper receipe for the given dish with accurate measurements if you can.

Outline structure:

- Dish: "Name of the dish"
- Ingredients: give the needed ingridients with measurements
- Step-by-Step Instructions on cooking
- add any general tips to improve the taste or best practis

**Parameters**: 
- dish: chicken biryani



> Entering new CrewAgentExecutor chain...
 Thought: Do I need to use a tool? Yes
Action: spoonacular_API.searchRecipes
Action Input: chicken biryaniERROR making requestParsing LLM output produced both a final answer and a parse-able action::  It looks like there was an error with the API request for searching recipes for "chicken biryani". Let's try a different approach and delegate this task to Master Chef, as they might have more knowledge about specific dishes.

Action: Delegate work to co-worker
Action Input: Master Chef | Find

/home/amaithi/projects/weaviate/langgraph-scratch/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/amaithi/projects/weaviate/langgraph-scratch/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


I attempted to call the Spoonacular Recipes API to search for a chicken biryani recipe based on your comment. However, the previous API call returned no results and zero total results, indicating that there is likely an issue with the data or parameters provided. I suggest checking the input for any errors before attempting another API call. It looks like there were issues with both Master Chef's response and the Spoonacular Recipes API in providing a valid receipe for "chicken biryani." To ensure accuracy, let's double-check the required ingredients and instructions by consulting a trusted recipe source, such as Allrecipes or Epicurious.

Action: Search on Allrecipes or Epicurious for Chicken Biryani Recipe
Action Input: "chicken biryani recipe"Search on Allrecipes or Epicurious for Chicken Biryani Recipe is not a valid tool, try one of [spoonacular_API.searchRecipes, spoonacular_API.searchRecipesByIngredients, spoonacular_API.searchRecipesByNutrients, spoonacular_API.getSimilarRecipe

multi agent

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

os.environ["OPENAI_API_KEY"] = ""


# Define Agents for Writing Crew
email_writer_1 = Agent(
    role='Concise Email Writer',
    goal='Write a short and engaging email',
    backstory='Experienced in writing concise marketing emails.',
    verbose=True,
    allow_delegation=True
)

email_writer_2 = Agent(
    role='Technical Email Writer',
    goal='Incorporate technical aspects of the video editing solution',
    backstory='Specializes in explaining technical products in an accessible way.',
    verbose=True,
    allow_delegation=False
)

email_writer_3 = Agent(
    role='Creative Email Writer',
    goal='Add a creative flair to the email',
    backstory='Brings a unique and creative perspective to marketing content.',
    verbose=True,
    allow_delegation=False
)



# Define Agents for Review Crew
proofreader = Agent(
    role='Proofreader',
    goal='Ensure grammatical correctness and clarity',
    backstory='Expert in English grammar and clarity in communication.',
    verbose=True,
    allow_delegation=False
    )

cold_email_specialist = Agent(
    role='Cold Email Specialist',
    goal='Provide strategic advice for cold emailing',
    backstory='Skilled in creating effective and engaging cold email strategies.',
    verbose=True,
    allow_delegation=True
)




# Define Agents for Critique Crew
dtc_cmo = Agent(
    role='DTC CMO',
    goal='Evaluate the email provided by writing_crew from a CMO perspective, modify the email to better fit the needs of a CMO, make sure to take into account the feedback provided by the review_crew. ',
    backstory='A CMO who frequently receives marketing emails.',
    verbose=True,
    allow_delegation=True
    
)

copywriter = Agent(
    role='Professional Copywriter',
    goal='Critique the email provided by writing_crew from a copywriting perspective and help dtc_cmo to rewrite the email to better fit the needs of a CMO.',
    backstory='A professional copywriter with extensive experience.',
    verbose=True,
    allow_delegation=False
)






In [ ]:
# Tasks

# Define Task for Writing Crew
email_writing_task = Task(
    description='Write three variations of a cold email selling a video editing solution. the solution is targeted towards marketing teams at DTC brands. the main unique selling point is the ability to create high-quality videos at scale. doubling down on ads that work and quickly iterating in order to test a large volume of ads without needing to wait for the video editors manually creating the videos.',
    agent=email_writer_1  # The task is assigned to the crew, not a specific agent
)


# Define Task for Critique Crew
critique_task = Task(
    description='Critique the cold emails provided by the writing_crew from prospects perspective, imagine you are recieving many emails proposing video editing services on a daily basis and you are looking for an exciting solution with very little risk. rewrite the 3 emails to better fit the needs of a CMO. DO TAKE INTO ACCOUNT the feedback provided by the review_crew.',
    agent=dtc_cmo
)

# Define Task for Review Crew
review_task = Task(
    description='Review and provide feedback on the cold emails provided by writing_crew.',
    agent=cold_email_specialist
)



In [51]:
# Crew


# Create Writing Crew
writing_crew = Crew(
    agents=[email_writer_1, email_writer_2, email_writer_3],
    tasks=[email_writing_task],
    verbose=True,
    process=Process.sequential
)


# Create Review Crew
review_crew = Crew(
    agents=[proofreader, cold_email_specialist],
    tasks=[review_task],
    verbose=True,
    process=Process.sequential
)


# Create Critique Crew
critique_crew = Crew(
    agents=[dtc_cmo, copywriter],
    tasks=[critique_task],
    verbose=True,
    process=Process.sequential
)


In [ ]:

# Execution Flow
print("Crew 1: Writing Cold Email Variations")
email_variations = writing_crew.kickoff()

print("\nCrew 2: Reviewing and Providing Feedback")
review_feedback = review_crew.kickoff()

print("\nCrew 3: Critiquing the Emails and rewriting them")
critique_feedback = critique_crew.kickoff()
